In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

1.[Reshape & pivot & melt](#Pivot&Stack)


   [pivot](#pivot)
   
   [pivot_table](#pivot_table)
   
  
   [melt](#melt)
   
   
   2.[working with text data](#series.str)
   
   [strip](#strip)
   
   [split](#split)
   
   [replace](#replace)
   
   3.[fillna](#fillna)

## Pivot&Stack


## 下面这个data他的index是有名字的，column也是有名字的！！！

In [ ]:
data= pd.DataFrame(np.arange(6).reshape((2, 3)),index=pd.Index(['Ohio', 'Colorado'], name='state'), columns=pd.Index(['one', 'two', 'three'], name='number'))

In [ ]:
data

## stack可以把列名，堆在row上

In [ ]:
result = data.stack()
result

## 用unstack可以恢复。

In [ ]:
result.unstack()

## 可以给函数传入我们想要他unstack的level

In [ ]:
result.unstack(0)

In [ ]:
result.unstack('state')

In [ ]:
result.unstack(1)

In [ ]:
result.unstack('number')

## nan的情况

In [ ]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

In [ ]:
data2.unstack()

In [ ]:
#stack回去，会自动赋值为0
data2.unstack().stack()

In [ ]:
#甚至可以增加一些row，通过dropna = False
data2.unstack().stack(dropna = False)

## 变化level的层次

## 先stack一下，把number放到row上，成为inner-most level

In [ ]:
result

In [ ]:
df = pd.DataFrame({'left': result, 'right': result + 5}, columns=pd.Index(['left', 'right'], name='side'))

In [ ]:
df

## 然后unstack state，再stack state!

In [ ]:
df.unstack(0)

In [ ]:
df.unstack(0).stack('state')

## pivot

In [ ]:
df = pd.DataFrame({'foo': ['one','one','one','two','two','two'],
                       'bar': ['A', 'B', 'C', 'A', 'B', 'C'],
                       'baz': [1, 2, 3, 4, 5, 6]})
df

In [ ]:
df.pivot(index = 'foo', columns = 'bar', values = 'baz')

In [ ]:
df.pivot(index = 'foo', columns = 'bar')

In [ ]:
df.pivot(index = 'foo', columns = 'bar')['baz']

## 其实pivot是一个set_index然后unstack的过程的简化

In [ ]:
df.set_index(['foo','bar'])

In [ ]:
df.set_index(['foo','bar']).unstack('bar')

## pivot_table

In [ ]:
df = pd.DataFrame({"A": ["foo", "foo", "foo", "foo", "foo",
...                          "bar", "bar", "bar", "bar"],
...                    "B": ["one", "one", "one", "two", "two",
...                          "one", "one", "two", "two"],
...                    "C": ["small", "large", "large", "small",
...                          "small", "large", "small", "small",
...                          "large"],
...                    "D": [1, 2, 2, 3, 3, 4, 5, 6, 7]})

In [ ]:
df

In [ ]:
table = pd.pivot_table(df, values = 'D', index = ['A','B'], columns = ['C'], aggfunc = np.sum)
table

## unstack

In [ ]:
index = pd.MultiIndex.from_tuples([('one', 'a'), ('one', 'b'),
...                                    ('two', 'a'), ('two', 'b')])

In [ ]:
index

In [ ]:
s = pd.Series(np.arange(1.0, 5.0), index=index)
s

In [ ]:
s.unstack(level=0)

In [ ]:
s.unstack(level=1)

## 默认一般使用-1，因为这是最里面一层的index，即last level,一般会拿来作为新的column嘛！


In [ ]:
s.unstack(level=-1)

## stack

这个和pivot差不多，就是增加一个index，一个level作为row。

In [ ]:
s = pd.DataFrame([[1,2],[3,4]], index = ['one','two'], columns = ['a','b'])

In [ ]:
s

In [ ]:
s.stack()

因为只有一个index，就默认stack到那里了，如果有多个，得注意用level = xx。

In [ ]:
df

## melt

melt就像是unpivot。把原先累加好的东西打散。

In [ ]:
df = pd.DataFrame({'A': {0: 'a', 1: 'b', 2: 'c'},
                    'B': {0: 1, 1: 3, 2: 5},                    'C': {0: 2, 1: 4, 2: 6}})

In [ ]:
df

In [ ]:
pd.melt(df, id_vars = ['A'], value_vars = ['B'])

In [ ]:
pd.melt(df, id_vars = ['A'], value_vars = ['B','C'])

还可以自定义新的column的名字

In [ ]:
pd.melt(df, id_vars = ['A'], value_vars = ['B'], var_name = "myVarname", value_name = "myValue")

如果有多层index的column

In [ ]:
df.columns = [list('ABC'),list('DEF')]

In [ ]:
df

In [ ]:
pd.melt(df, col_level = 0, id_vars = ['A'], value_vars = ['B'])

In [ ]:
pd.melt(df, id_vars = [('A', 'D')], value_vars = [('B','E')])

## series.str

In [ ]:
s = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca', np.nan, 'CABA', 'dog', 'cat'])

In [ ]:
s.str.upper()

In [ ]:
s.str.len()

## 可以实现一些string的操作！

## strip

In [ ]:
idx = pd.Index([' jack', 'jill ', ' jesse ', 'frank'])

In [ ]:
idx.str.strip()

In [ ]:
idx.str.lstrip()

In [ ]:
idx.str.rstrip()

## 这些操作对于清理一些数据的字符串很有帮助，例如有空格啊，大小写啊，“_"分隔啊，什么的，可以通过这些函数把格式统一掉。

## split

In [ ]:
s2 = pd.Series(['a_b_c', 'c_d_e', np.nan, 'f_g_h'])

In [ ]:
s2.str.split("_")

## 可以获取一列的数据，通过str[]或者str.get()

In [ ]:
s2.str.split("_").str[1]

In [ ]:
s2.str.split("_").str.get(1)

## 把这个series变成dataframe

In [ ]:
s2.str.split("_", expand = True)

## 还可以控制split的个数

In [ ]:
s2.str.split("_", expand = True, n = 1)

## replace
是用正则表达式去替换。

In [ ]:
s3 = pd.Series(['A', 'B', 'C', 'Aaba', 'Baca','', np.nan, 'CABA', 'dog', 'cat'])

In [ ]:
s3

In [ ]:
s3.str.replace('^.a|dog','xx-xx', case = False)

## 也可以单纯的代替！让regex = False即可

In [ ]:
dollars = pd.Series(['12', '-$10', '$10,000'])

In [ ]:
dollars.str.replace('-$','-', regex = False)

## fillna

In [ ]:
df = pd.DataFrame([[np.nan,2,np.nan,0],
                   [3,4,np.nan,1],
                   [np.nan,np.nan,np.nan,5],
                   [np.nan,3,np.nan,4]],
                   columns=list('ABCD'))


In [ ]:
df

In [ ]:
df.fillna(0)

In [ ]:
df.B.fillna(np.mean(df.B))

## ffill是拿前一个数来fill自己
pad 和ffill是一样的

In [ ]:
df.fillna(method = 'ffill')

In [ ]:
df.fillna(method = 'bfill')

## 把axis改为1，那么就会按行去填充

In [ ]:
df.fillna(method = 'bfill',axis = 1)

In [ ]:
df.fillna(method = 'pad')